In [1]:
!pip install flask datetime jugaad_data pandas ta numpy yfinance scikit-learn tensorflow Werkzeug==2.0.3 Flask==2.0.3


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of yfinance to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of yfinance to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.2/289.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.8/115.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from flask import Flask, render_template, request, redirect, url_for, jsonify
import datetime as dt
from jugaad_data.nse import NSELive,stock_df
import pandas as pd
import ta
from datetime import date, timedelta,datetime,time
import numpy as np
import yfinance as yf
import csv
# from ta.momentum import RSIIndicator, StochasticOscillator
# from ta.trend import MACD, EMAIndicator, SMAIndicator, ADXIndicator
# from ta.volatility import BollingerBands
import json
import warnings
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from functools import lru_cache
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

#from news import merger
#final_decision_news = merger()

def predict(df):


    if df.empty:
        return "No data found for the given stock symbol."

    # Preprocess the data
    df['Close'] = df['Close'].fillna(method='ffill')  # Handle NaNs
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))

    # Prepare training data
    X_train = []
    y_train = []
    window_size = 60  # The number of days for the input sequence

    # Check if `scaled_data` has enough data points
    if len(scaled_data) < window_size:
        print("Not enough data to create training samples. Skipping this stock.")
        return 0

    # Build `X_train` and `y_train`
    for i in range(window_size, len(scaled_data)):
        X_train.append(scaled_data[i-window_size:i, 0])
        y_train.append(scaled_data[i, 0])

    # Check if `X_train` is empty
    if not X_train:
        print("X_train is empty. Skipping this stock.")
        return "No training data prepared."

    # Convert lists to numpy arrays and reshape `X_train`
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

    # Build LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=10, batch_size=32)

    # Prepare data for prediction
    last_60_days = scaled_data[-60:]
    last_60_days = last_60_days.reshape((1, last_60_days.shape[0], 1))

    # Predict the next day's price
    predicted_price_scaled = model.predict(last_60_days)
    predicted_price = scaler.inverse_transform(predicted_price_scaled)

    predicted_price = round(predicted_price[0][0], 2)
    return predicted_price


def trendlyne():

    #csv_file = 'merged_output.csv'
    csv_file = '/content/drive/MyDrive/Colab Notebooks/nifty50/merged_output.csv'
    df_csv = pd.read_csv(csv_file)
    df_csv.columns = df_csv.columns.str.strip()
    df_csv = df_csv.dropna(subset=[df_csv.columns[0]], how='all')
    final_decision_news = df_csv
    return final_decision_news






###########################################################################
nse = NSELive()
# Caching stock data fetching
#@lru_cache(maxsize=1000)
def fetch_delivery_data(symbols, num1):
    print("symbols data fetching in progress...")
    all_data = {}
    for symbol in symbols:

        stock = yf.Ticker(symbol)
        if num1 == 0:
            df = stock.history(period="1mo", interval="15m") # 30d 1h identifies Stocks for delivery
        else:
            df = stock.history(period="5d", interval="15m") # 7d 1m identifies Stocks for intraday

        df['pe'] = stock.info.get('trailingPE')
        df['eps'] = stock.info.get('trailingEps')

        if not df.empty:
            checkpe = df['pe'].iloc[-1]
            checkeps = df['eps'].iloc[-1]
            if checkpe is None:
                df['pe'] =0
            if checkeps is None:
                df['eps'] = 0

        if not df.empty:
            all_data[symbol] = df
    print("symbols data fetching completed.")
    return all_data

# New function to handle parallel processing
def fetch_delivery_data_parallel(symbols, num1):
    all_data = {}
    print("symbols data fetching in progress...")
    with ThreadPoolExecutor(max_workers=5) as executor:
        # Create a future for each symbol
        future_to_symbol = {executor.submit(fetch_single_symbol_data, symbol, num1): symbol for symbol in symbols}

        for future in concurrent.futures.as_completed(future_to_symbol):
            symbol = future_to_symbol[future]
            try:
                data = future.result()
                if data is not None:
                    all_data[symbol] = data
            except Exception as e:
                print(f"Error fetching data for {symbol}: {e}")
    print("symbols data fetching Completed.")
    return all_data

# Helper function to fetch data for a single symbol
def fetch_single_symbol_data(symbol, num1):
    stock = yf.Ticker(symbol)
    if num1 == 0:
        df = stock.history(period="1mo", interval="15m")
    else:
        df = stock.history(period="5d", interval="15m")

    df['pe'] = stock.info.get('trailingPE')
    df['eps'] = stock.info.get('trailingEps')

    if not df.empty:
        checkpe = df['pe'].iloc[-1]
        checkeps = df['eps'].iloc[-1]
        if checkpe is None:
            df['pe'] = 0
        if checkeps is None:
            df['eps'] = 0

    return df if not df.empty else None




def calculate_vix(symb):

    return 0,0,0


# Dummy symbols data for demonstration
predefined_symbols123 = [  "ADANIENT","ADANIPORTS", "APOLLOHOSP","ASIANPAINT", "AXISBANK", "BAJAJ-AUTO", "BAJAJFINSV",
    "BAJFINANCE","BRITANNIA", "BHARTIARTL", "BPCL", "CIPLA", "COALINDIA", "DIVISLAB",
    "DRREDDY", "EICHERMOT", "GRASIM", "HCLTECH","HDFCBANK",
    "HDFCLIFE", "HEROMOTOCO", "HINDALCO", "HINDUNILVR", "ICICIBANK",
    "INDUSINDBK", "INFY", "IOC", "ITC", "JSWSTEEL", "KOTAKBANK", "LTIM","LT",
    "M&M", "MARUTI", "NESTLEIND", "NTPC", "ONGC", "POWERGRID", "RELIANCE",
    "SBILIFE", "SBIN", "SHREECEM", "SUNPHARMA", "TATAMOTORS", "TATASTEEL",
    "TCS", "TECHM", "TITAN", "ULTRACEMCO", "UPL", "WIPRO"
   ]
predefined_symbols_s = ["360ONE.NS", "AARTIIND.NS", "AAVAS.NS", "ACE.NS", "AETHER.NS", "AFFLE.NS",     "APLLTD.NS", "ALKYLAMINE.NS", "ALLCARGO.NS", "ALOKINDS.NS", "ARE&M.NS", "AMBER.NS",     "ANANDRATHI.NS", "ANGELONE.NS", "ANURAS.NS", "APARINDS.NS", "APTUS.NS", "ACI.NS",     "ASAHIINDIA.NS", "ASTERDM.NS", "ASTRAZEN.NS", "AVANTIFEED.NS", "BEML.NS", "BLS.NS",     "BALAMINES.NS", "BALRAMCHIN.NS", "BIKAJI.NS", "BIRLACORPN.NS", "BSOFT.NS", "BLUEDART.NS",     "BLUESTARCO.NS", "BBTC.NS", "BORORENEW.NS", "BRIGADE.NS", "MAPMYINDIA.NS", "CCL.NS",     "CESC.NS", "CIEINDIA.NS", "CSBBANK.NS", "CAMPUS.NS", "CANFINHOME.NS", "CAPLIPOINT.NS",     "CGCL.NS", "CASTROLIND.NS", "CEATLTD.NS", "CELLO.NS", "CENTRALBK.NS", "CDSL.NS",     "CENTURYPLY.NS", "CENTURYTEX.NS", "CERA.NS", "CHALET.NS", "CHAMBLFERT.NS", "CHEMPLASTS.NS",     "CHENNPETRO.NS", "CHOLAHLDNG.NS", "CUB.NS", "CLEAN.NS", "COCHINSHIP.NS", "CAMS.NS",     "CONCORDBIO.NS", "CRAFTSMAN.NS", "CREDITACC.NS", "CROMPTON.NS", "CYIENT.NS", "DCMSHRIRAM.NS",     "DOMS.NS", "DATAPATTNS.NS", "DEEPAKFERT.NS", "EIDPARRY.NS", "EIHOTEL.NS", "EPL.NS",     "EASEMYTRIP.NS", "ELECON.NS", "ELGIEQUIP.NS", "ENGINERSIN.NS", "EQUITASBNK.NS", "ERIS.NS",     "EXIDEIND.NS", "FDC.NS", "FINEORG.NS", "FINCABLES.NS", "FINPIPE.NS", "FSL.NS", "FIVESTAR.NS",     "GMMPFAUDLR.NS", "GRSE.NS", "GILLETTE.NS", "GLS.NS", "GLENMARK.NS", "MEDANTA.NS", "GPIL.NS",     "GODFRYPHLP.NS", "GRANULES.NS", "GRAPHITE.NS", "GESHIP.NS", "GAEL.NS", "GMDCLTD.NS",     "GNFC.NS", "GPPL.NS", "GSFC.NS", "GSPL.NS", "HEG.NS", "HFCL.NS",     "HAPPSTMNDS.NS", "HAPPYFORGE.NS", "HSCL.NS", "HINDCOPPER.NS", "POWERINDIA.NS", "HOMEFIRST.NS",     "HONASA.NS", "HUDCO.NS", "IDFC.NS", "IIFL.NS", "IRB.NS", "IRCON.NS", "ITI.NS",     "INDIACEM.NS", "INDIAMART.NS", "IEX.NS", "IOB.NS", "INDIGOPNTS.NS", "INOXWIND.NS", "INTELLECT.NS",     "JBCHEPHARM.NS", "JBMA.NS", "JKLAKSHMI.NS", "JKPAPER.NS", "JMFINANCIL.NS", "JAIBALAJI.NS",     "J&KBANK.NS", "JINDALSAW.NS", "JUBLINGREA.NS", "JUBLPHARMA.NS", "JWL.NS", "JUSTDIAL.NS",     "JYOTHYLAB.NS", "KNRCON.NS", "KRBL.NS", "KSB.NS", "KPIL.NS", "KARURVYSYA.NS", "KAYNES.NS",     "KEC.NS", "KFINTECH.NS", "KIMS.NS", "LATENTVIEW.NS", "LXCHEM.NS", "LEMONTREE.NS", "MMTC.NS",     "MTARTECH.NS", "MGL.NS", "MAHSEAMLES.NS", "MHRIL.NS", "MAHLIFE.NS", "MANAPPURAM.NS",     "MRPL.NS", "MASTEK.NS", "MEDPLUS.NS", "METROPOLIS.NS", "MINDACORP.NS", "MOTILALOFS.NS",     "MCX.NS", "NATCOPHARM.NS", "NBCC.NS", "NCC.NS", "NLCINDIA.NS", "NSLNISP.NS", "NH.NS",     "NATIONALUM.NS", "NAVINFLUOR.NS", "NETWORK18.NS", "NAM-INDIA.NS", "NUVAMA.NS", "NUVOCO.NS",     "OLECTRA.NS", "PCBL.NS", "PNBHOUSING.NS", "PNCINFRA.NS", "PVRINOX.NS", "PPLPHARMA.NS",     "POLYMED.NS", "PRAJIND.NS", "PRINCEPIPE.NS", "PRSMJOHNSN.NS", "QUESS.NS", "RRKABEL.NS",     "RBLBANK.NS", "RHIM.NS", "RITES.NS", "RADICO.NS", "RAILTEL.NS", "RAINBOW.NS",     "RAJESHEXPO.NS", "RKFORGE.NS", "RCF.NS", "RATNAMANI.NS", "RTNINDIA.NS", "RAYMOND.NS",     "REDINGTON.NS", "RBA.NS", "ROUTE.NS", "SBFC.NS", "SAFARI.NS", "SAMMAANCAP.NS",     "SANOFI.NS", "SAPPHIRE.NS", "SAREGAMA.NS", "SCHNEIDER.NS", "RENUKA.NS", "SHYAMMETL.NS",     "SIGNATURE.NS", "SOBHA.NS", "SONATSOFTW.NS", "SWSOLAR.NS", "STLTECH.NS", "SPARC.NS",     "SUNTECK.NS", "SUVENPHAR.NS", "SWANENERGY.NS", "SYRMA.NS", "TV18BRDCST.NS", "TVSSCS.NS",     "TMB.NS", "TANLA.NS", "TATAINVEST.NS", "TTML.NS", "TEJASNET.NS", "TITAGARH.NS",     "TRIDENT.NS", "TRIVENI.NS", "TRITURBINE.NS", "UCOBANK.NS", "UTIAMC.NS", "UJJIVANSFB.NS",     "USHAMART.NS", "VGUARD.NS", "VIPIND.NS", "VAIBHAVGBL.NS", "VTL.NS", "VARROC.NS",     "VIJAYA.NS", "WELCORP.NS", "WELSPUNLIV.NS", "WESTLIFE.NS", "WHIRLPOOL.NS", "ZENSARTECH.NS",     "ECLERX.NS"]

predefined_symbols = [f"{symbol}.NS" for symbol in predefined_symbols123]

predefined_symbols_5 = [
    "SCI", "ITI", "PFC", "IFCI", "ENDURANCE", "NAUKRI", "BIOCON", "GVT&D", "RECLTD", "JSWENERGY",
    "POWERGRID", "PNBHOUSING", "EMCURE", "PRESTIGE", "ECLERX", "EIHOTEL", "KIMS", "GLAND", "BLS",
    "BBTC", "TRENT", "OFSS", "BAJAJHLDNG", "SCHAEFFLER", "ENGINERSIN", "TEJASNET", "ASHOKLEY",
    "TATAINVEST", "MRF", "SUPREMEIND", "KAYNES", "ABFRL", "TATAMOTORS", "THERMAX", "UNITDSPR",
    "BANKINDIA", "GICRE", "WIPRO", "ABB", "DOMS", "MANAPPURAM", "ANANDRATHI", "LODHA", "SIGNATURE",
    "SWANENERGY", "LEMONTREE", "NEWGEN", "METROBRAND", "TCS", "TECHM", "SOLARINDS", "KALYANKJIL",
    "LICI", "CRISIL", "HCLTECH", "INFY", "JSWINFRA", "HAL", "SOBHA", "FINEORG", "BEL", "PTCIL",
    "HDFCBANK", "ANANTRAJ", "MARUTI", "NIACL", "CAMS", "SHRIRAMFIN", "LTIM", "EMAMILTD", "CGPOWER",
    "INDIANB", "LTF", "MEDANTA", "IDFCFIRSTB", "JUBLFOOD", "PNCINFRA", "RKFORGE", "PERSISTENT",
    "360ONE", "ICICIBANK", "IRCTC", "BALKRISIND", "METROPOLIS", "DMART", "BSOFT", "CAMPUS",
    "MASTEK", "LTTS", "IRFC", "AXISBANK", "BHEL", "TBOTEK", "BAJAJ-AUTO", "CHALET", "TATACOMM",
    "GODIGIT", "CANBK", "SBIN", "TITAN", "COFORGE", "MOTHERSON", "JUSTDIAL", "BANKBARODA",
    "GODREJPROP", "VARROC", "IEX", "INDIGO", "ZOMATO", "PVRINOX", "MUTHOOTFIN", "TVSMOTOR",
    "EXIDEIND", "GLENMARK", "TIMKEN", "GAIL", "HFCL", "KOTAKBANK", "ABBOTINDIA", "MPHASIS",
    "IDBI", "CCL", "TATAPOWER", "CESC", "HUDCO", "OBEROIRLTY", "UNIONBANK", "NH", "BOSCHLTD",
    "BPCL", "LLOYDSME", "TORNTPOWER", "GRAPHITE", "FORTIS", "CENTRALBK", "PCBL", "POLYCAB",
    "MMTC", "NETWEB", "HINDPETRO", "KARURVYSYA", "ABCAPITAL", "APLAPOLLO", "ERIS", "SBFC",
    "REDINGTON", "DBREALTY", "CHOLAFIN", "SYRMA", "IRB", "ADANIENT", "ISEC", "INDIAMART",
    "VIPIND", "INDUSINDBK", "ICICIGI", "BAJAJFINSV", "DRREDDY", "DLF", "KPIL", "VEDL", "EICHERMOT",
    "IREDA", "BDL", "SONACOMS", "USHAMART", "GRANULES", "KIRLOSBROS", "BEML", "GPPL", "PIIND",
    "ITC", "CHAMBLFERT", "BHARTIARTL", "JKTYRE", "ASTERDM", "IOB", "MCX", "HDFCLIFE", "ESCORTS",
    "GESHIP", "INOXINDIA", "ULTRACEMCO", "TITAGARH", "UBL", "KEI", "AMBUJACEM", "YESBANK",
    "COALINDIA", "PATANJALI", "PNB", "FIVESTAR", "GSPL", "IDEA", "ZYDUSLIFE", "COLPAL", "GUJGASLTD",
    "TVSSCS", "TATACONSUM", "RTNINDIA", "VOLTAS", "VBL", "BIKAJI", "JINDALSAW", "IIFL", "MAHABANK",
    "CREDITACC", "FEDERALBNK", "ZENSARTECH", "GLAXO", "KSB", "TTML", "GAEL", "MFSL", "SKFINDIA",
    "KAJARIACER", "HONAUT", "PRAJIND", "SBILIFE", "COROMANDEL", "MANKIND", "INDHOTEL", "KIRLOSENG",
    "BLUEDART", "SYNGENE", "JPPOWER", "BIRLACORPN", "FINCABLES", "ICICIPRULI", "BSE", "HINDALCO",
    "SUNPHARMA", "HDFCAMC", "NATCOPHARM", "AVANTIFEED", "NAM-INDIA", "IOC", "RCF", "RELIANCE",
    "ASAHIINDIA", "RAILTEL", "GRINFRA", "AEGISLOG", "PETRONET", "KNRCON", "CAPLIPOINT", "NTPC",
    "ACC", "MANYAVAR", "UTIAMC", "BATAINDIA", "AAVAS", "JUBLPHARMA", "MINDACORP", "SBICARD",
    "FLUOROCHEM", "LT", "HEROMOTOCO", "BLUESTARCO", "CONCORDBIO", "GRASIM", "BALAMINES",
    "INDUSTOWER", "CASTROLIND", "HONASA", "PIDILITIND", "CONCOR", "JIOFIN", "HINDUNILVR",
    "SIEMENS", "CROMPTON", "APOLLOTYRE", "ABSLAMC", "ZFCVINDIA", "RAMCOCEM", "NESTLEIND",
    "TATATECH", "DEVYANI", "SCHNEIDER", "HAVELLS", "GMRINFRA", "UCOBANK", "KPRMILL", "KPITTECH",
    "SPARC", "JUBLINGREA", "ABREL", "ALKEM", "TANLA", "ASTRAZEN", "JKCEMENT", "RAINBOW", "ELECON",
    "POONAWALLA", "PHOENIXLTD", "CANFINHOME", "OIL", "POLYMED", "APLLTD", "ADANIPORTS", "TORNTPHARM",
    "HAPPSTMNDS", "CRAFTSMAN", "QUESS", "BAJFINANCE", "ARE&M", "WELSPUNLIV", "SHREECEM", "MRPL",
    "ALOKINDS", "LUPIN", "CYIENT", "IGL", "GRINDWELL", "CIEINDIA", "CHENNPETRO", "ZEEL", "CEATLTD",
    "DALBHARAT", "BANDHANBNK", "UNOMINDA", "POLICYBZR", "TATACHEM", "NAVINFLUOR", "ROUTE", "GNFC",
    "GILLETTE", "DELHIVERY", "AIAENG", "AUBANK", "CARBORUNIV", "GODREJIND", "WELCORP", "JSWSTEEL",
    "ADANIGREEN", "BRITANNIA", "TRIVENI", "NYKAA", "DIVISLAB", "J&KBANK", "CERA", "GODFRYPHLP",
    "AJANTPHARM", "BHARTIHEXA", "AMBER", "PGHH", "HINDCOPPER", "ATGL", "MGL", "MARICO", "HSCL",
    "ADANIPOWER", "DIXON", "LALPATHLAB", "OLECTRA", "SWSOLAR", "ACE", "JINDALSTEL", "GPIL", "SJVN",
    "AFFLE", "ONGC", "SUMICHEM", "PAGEIND", "NETWORK18", "SAIL", "M&M", "MAZDOCK", "NCC", "APOLLOHOSP",
    "RATNAMANI", "GODREJAGRO", "DABUR", "HINDZINC", "TECHNOE", "INDIACEM", "KFINTECH", "UNIPARTS",
    "TATVA", "RELIGARE", "DEVHARI", "JSWHL", "KAPSTON", "CHOTHANI", "GOLDCASH", "TATVAKRL"
]
predefined_symbols_500 = [f"{symbol}.NS" for symbol in predefined_symbols_5]
predefined_symbols_m =["ACC.NS", "APLAPOLLO.NS", "AUBANK.NS", "ABCAPITAL.NS", "ABFRL.NS", "ALKEM.NS", "APOLLOTYRE.NS", "ASHOKLEY.NS", "ASTRAL.NS", "AUROPHARMA.NS", "BSE.NS", "BALKRISIND.NS", "BANDHANBNK.NS", "BANKINDIA.NS", "MAHABANK.NS", "BDL.NS", "BHARATFORG.NS", "BHARTIHEXA.NS", "BIOCON.NS", "CGPOWER.NS", "COCHINSHIP.NS", "COFORGE.NS", "COLPAL.NS", "CONCOR.NS", "CUMMINSIND.NS", "DELHIVERY.NS", "DIXON.NS", "ESCORTS.NS", "EXIDEIND.NS", "NYKAA.NS", "FEDERALBNK.NS", "FACT.NS", "GMRINFRA.NS", "GODREJPROP.NS", "HDFCAMC.NS", "HINDPETRO.NS", "HINDZINC.NS", "HUDCO.NS", "IDBI.NS", "IDFCFIRSTB.NS", "IRB.NS", "INDIANB.NS", "INDHOTEL.NS", "IOB.NS", "IREDA.NS", "IGL.NS", "INDUSTOWER.NS", "JSWINFRA.NS", "JUBLFOOD.NS", "KPITTECH.NS", "KALYANKJIL.NS", "LTF.NS", "LICHSGFIN.NS", "LUPIN.NS", "MRF.NS", "M&MFIN.NS", "MRPL.NS", "MANKIND.NS", "MARICO.NS", "MFSL.NS", "MAXHEALTH.NS", "MAZDOCK.NS", "MPHASIS.NS", "MUTHOOTFIN.NS", "NLCINDIA.NS", "NMDC.NS", "OBEROIRLTY.NS", "OIL.NS", "PAYTM.NS", "OFSS.NS", "POLICYBZR.NS", "PIIND.NS", "PAGEIND.NS", "PATANJALI.NS", "PERSISTENT.NS", "PETRONET.NS", "PHOENIXLTD.NS", "POLYCAB.NS", "POONAWALLA.NS", "PRESTIGE.NS", "RVNL.NS", "SBICARD.NS", "SJVN.NS", "SRF.NS", "SOLARINDS.NS", "SONACOMS.NS", "SAIL.NS", "SUNDARMFIN.NS", "SUPREMEIND.NS", "SUZLON.NS", "TATACHEM.NS", "TATACOMM.NS", "TATAELXSI.NS", "TATATECH.NS", "TORNTPOWER.NS", "TIINDIA.NS", "UPL.NS", "IDEA.NS", "VOLTAS.NS", "YESBANK.NS"]

#tech superbuy symbols
def tech_superbuy():
    #df_tech = pd.read_excel('auto_updated_with_decisions.xlsx')  # Adjust sheet name if necessary

    final_decision_news = trendlyne()


    results1 = []
    for index, row in final_decision_news.iterrows():
        #tech_stock_symbol = row['Stock Symbol']
        #results1.append(tech_stock_symbol)

        tech_stock_symbol =row['Stock Symbol']
        news_symb1 =tech_stock_symbol +".NS"

        news_tech = row['Final Output']




        if news_tech =='SuperBuy' or news_tech =='IntraBuy'or news_tech =='Buy' or news_tech =='Watch':
            results1.append(news_symb1)

    return results1







def delivery():



    symbols1 = '500' #input("Enter symbol:")

    if symbols1 == 'small':
        symbols =predefined_symbols_s
        #symbols = symbols1 if symbols1 else predefined_symbols
    elif symbols1 == 'nifty50':
        symbols =predefined_symbols
    elif symbols1 == '500':
        symbols = predefined_symbols_500
    elif symbols1 == 'mid':
        symbols = predefined_symbols_m

    elif symbols1 == 'superbuy':

        symbols = tech_superbuy()

    else:
        symbols = symbols1.split(',') if symbols1 else predefined_symbols


    last_refreshed = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    data = fetch_delivery_data_parallel(symbols,0)

    _,vix,vix_senti = calculate_vix('^INDIAVIX')
    results = []
    now = 0
    i=1
    # if symbols_get == "":
    #     if symbols1 == 'superbuy':
    final_decision_news1 = trendlyne()


    for symbol, df in data.items():
        try:

            # Calculate indicators
            for index, row in final_decision_news1.iterrows():

                news_symb =row['Stock Symbol']
                news_symb1 =news_symb +".NS"

                if symbol == news_symb1:
                    news_decision_t = row['Final Output']

                    #news_decision_pcr = row['Final Decision']
                    break
                else:
                    news_decision_t = 'Hold'
                    #news_decision_pcr = 'Hold'

            news_tech = news_decision_t
            #news_pcr = news_decision_pcr
            news_pcr = 'Hold'
            #if news_tech =='SuperBuy':

            #df = calculate_indicators(df,now)

            df['time'] = df.index.time

            df['date'] = df.index.date
            # Replace 'df' with your DataFrame variable
            # Get the current time and date
            now = datetime.now()
            new_datetime = now + timedelta(hours=5, minutes=30)
            current_time = new_datetime.time()
            today = now.date()

           # If the time is before 9.30am today then will pick yesterday's close price else today's last 15m candle close price.
            if current_time < time(9, 30):
                # Select the close price at 3:30 PM of the previous day
                previous_day_data = df[(df['date'] < today) & (df['time'] <= time(15, 30))].sort_values(by=['date', 'time'], ascending=[False, False])

                if not previous_day_data.empty:
                    CP_result = previous_day_data.iloc[0]['Close']
                    CP_open = previous_day_data.iloc[0]['Close']
                else:
                    CP_result = 1
                    print("No 3:30 PM data found for previous days.")
            else:
                # Select the close price at 3:30 PM or the most recent time up to 3:30 PM of today
                today_data = df[(df['time'] <= time(15, 30)) & (df['date'] == today)].sort_values(by='time', ascending=True)

                if not today_data.empty:
                    CP_result = today_data.iloc[-1]['Close']
                    CP_open = today_data.iloc[0]['Open']
                else:
                    CP_result = 1
                    print("No 3:30 PM data found for today.")

            #print("Selected Close price:", CP_result)
            LTP_percentage = (CP_result - CP_open)/CP_open*100
            predicted_price = predict(df)
            LTP_percentage = round(LTP_percentage,2)
            CP_result  = round(CP_result,2)

            percent = (predicted_price - CP_result)/CP_result*100
            percent= round(percent,2)
            if predicted_price >= CP_result:
                if percent >= 1.2 and LTP_percentage >0.5:
                    decision = "Predicted Buy"
                elif percent >= 1.2 and LTP_percentage >-1:
                    decision = "Predicted Buy Opportunity"

                else:
                    decision = "Predicted Watch"

            else:

                decision = "Predicted Sell"


            last_Price = CP_result


            symbols_NS = symbol[:-3]


            print(i,symbol, decision)
            i=i+1
            # Prepare data for each symbol
            symbol_data = {
                'symbol': symbol,

                'LTP': last_Price, #close price
                'pChange':percent,
                'change':LTP_percentage,
                'decision': decision,

                'predicted_price':predicted_price



            }

            results.append(symbol_data)

        except KeyError as e:
            print(f"KeyError: {str(e)}. Skipping symbol {symbol}.")
            continue

    # Specify the header for the CSV file (the keys from the first dictionary in `results`)
    header = ['symbol', 'LTP', 'change','predicted_price','pChange', 'decision', ]

    # Define the file path
    #csv_file_path = 'predictML.csv'
    csv_file_path = '/content/drive/MyDrive/Colab Notebooks/nifty50/predictML1.csv'

    # Open the file in write mode and create a CSV writer
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)

        # Write the header row
        writer.writeheader()

        # Write the rows (data) from the `results` list
        writer.writerows(results)

    print(f"Results saved to {csv_file_path}")
    return "completed."



if __name__ == "__main__":
    delivery()

    #app.run(debug=True, host='0.0.0.0', port=80)

symbols data fetching in progress...
- DEVHARI.NS: No data found, symbol may be delisted
- CHOTHANI.NS: No data found, symbol may be delisted
- GOLDCASH.NS: No data found, symbol may be delisted
- TATVAKRL.NS: No data found, symbol may be delisted
symbols data fetching Completed.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.2580
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0191
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0121
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0078
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0063
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0059
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0057
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0048
Epoch 9/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0055
Epoch 10/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0051
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1 IFCI.NS Predicted Sell
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1115
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0128
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0054
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0030
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0026
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0022
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0019
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0020
Epoch 9/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0022
Epoch 10/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0018
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
2 ITI.NS Predicted Sell
Epoch 1/10


...

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2663
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0287
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0125
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0088
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0078
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0063
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0057
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0054
Epoch 9/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0055
Epoch 10/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0057
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
380 UNIPARTS.NS Predicted Watch
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0732
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0117
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0024
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0017
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0016
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0014
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0013
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0012
Epoch 9/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0011
Epoch 10/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7.3973e-04
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
381 JSWHL.NS Predicted Sell
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.2664
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0165
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0110
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0085
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0062
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0049
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0054
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0055
Epoch 9/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0042
Epoch 10/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0047
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
382 KAPSTON.NS Predicted Buy
Results saved to /content/drive/MyDrive/Colab Notebooks/nifty50/predictML1.csv
